<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/5_Datos_externos_bomberos_Madrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datos externos de geolocalización

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time

In [ ]:
!git clone https://github.com/franciscogarate/cdiae

En este ejemplo, usaremos la librería de acceso libre y gratuita Nominatim de OpenStreetMaps. No obstante, no siempre es fiable y por ello la API recomendada es GoogleV3 (la cual necesita un token de pago).

`from geopy.geocoders import GoogleV3`

`geolocator = GoogleV3(api_key=***)`

In [ ]:
def obtener_coordenadas(address):
    geolocator = Nominatim(user_agent="Tu-App/1.0")
    location = geolocator.geocode(address)
    time.sleep(1)               # Pausa de 1 segundo para evitar denegación de servicio
    if location:
        return location.latitude, location.longitude
    else:
        return None, None


In [ ]:
df = pd.read_csv('cdiae/data/01_raw/211642-1-bomberos-parques.csv', sep=';', usecols=(9,10,12))
df.head()

In [ ]:
df['direccion'] = df.apply(lambda x: str(x['CLASE-VIAL'] + ' ' + x['NOMBRE-VIA']) + ', ' + str(x.NUM) + ', Madrid, Spain', axis=1)
df['coordenadas'] = df['direccion'].apply(obtener_coordenadas)
df

Guardo las coordenadas en un array con numpy:

In [ ]:
coord_riesgos = np.asarray(df.coordenadas)

### Grafico con folium

Primero, centro el mapa en Madrid

In [ ]:
from folium import Map, plugins
from folium.plugins import HeatMap
map_madrid = Map(location=[40.42, -3.70], zoom_start = 12)

Muestro el mapa de calor

In [ ]:
coord_riesgos = np.asarray(df.coordenadas)
HeatMap(coord_riesgos, radius=20).add_to(map_madrid)
map_madrid